# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Vector DB - BigQuery using Custom Embedding model.

* This notebook explains how to use BigQuery as a Vector database using Custom Embedding model.
* Reference
  * https://cloud.google.com/bigquery/docs/vector-search-intro?hl=ko
  * https://python.langchain.com/docs/integrations/vectorstores/google_bigquery_vector_search
  * https://api.python.langchain.com/en/stable/embeddings/langchain_core.embeddings.Embeddings.html#langchain_core.embeddings.Embeddings

* Data reference
  * https://www.data.go.kr/data/15069932/fileData.do

# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [ ]:
%pip install --upgrade --quiet google-cloud-aiplatform

In [ ]:
%pip install --upgrade --quiet langchain langchain-community

In [ ]:
%pip install --upgrade --quiet sentence_transformers

In [ ]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [ ]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

* Mount to the google drive to access the .ipynb files in the repository.



In [ ]:
# To access contents in Google drive

if "google.colab" in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')

# Execute the example
## Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [ ]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [ ]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)

# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

## Custom Embedding
* To use Langchain function for custom embedding, you need to implement Ebeddings interface like the following steps.

* Refer to Ebeddings interface implementation : https://api.python.langchain.com/en/stable/embeddings/langchain_core.embeddings.Embeddings.html#langchain_core.embeddings.Embeddings

* Need to implement the following two functions
  * abstract embed_documents(texts: List[str]) → List[List[float]]
  * embed_query(text: str) → List[float]

In [ ]:
from langchain_core.embeddings import Embeddings
from typing import List

from sentence_transformers import SentenceTransformer

class Custom_Embedding(Embeddings):

  model = None

  def __init__(self, model_name: str):
    self.model = SentenceTransformer(model_name)

  def embed_documents(self, texts: List[str]) -> List[List[float]]:
    embeddings = self.model.encode(texts)
    return embeddings.tolist()

  def embed_query(self, text: str) -> List[float]:
    embeddings = self.model.encode([text])
    return embeddings.tolist()[0]

## Custom Embeddings model
* Use a model that was verified in Hugging face
* Use your model for the specific purpose if you have your model.

  * https://huggingface.co/snunlp/KR-SBERT-V40K-klueNLI-augSTS
  * https://huggingface.co/sentence-transformers/stsb-xlm-r-multilingual

In [ ]:
EBEDDING_MODEL = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
embedding = Custom_Embedding(EBEDDING_MODEL)

## Create dataset and table in BigQuery

In [ ]:
from google.cloud import bigquery

DATASET = "vector_db_custom"
TABLE = "vector_table_custom"

client = bigquery.Client(project=PROJECT_ID, location=REGION)
client.create_dataset(dataset=DATASET, exists_ok=True)


## Vector Search environment

In [ ]:

from langchain.vectorstores.utils import DistanceStrategy
from langchain_community.vectorstores import BigQueryVectorSearch

table = BigQueryVectorSearch(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,

    #https://api.python.langchain.com/en/stable/vectorstores/langchain_community.vectorstores.utils.DistanceStrategy.html#langchain_community.vectorstores.utils.DistanceStrategy
    distance_strategy=DistanceStrategy.COSINE

)

## Read the text data that will be vectorized.

In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/projects/google_gen_ai_sample/contents/text/legal_terminology.csv'

legal_terms = pd.read_csv(file_path,sep=",", encoding='utf-8-sig')
legal_terms

## Embedding and store the vector data into BigQuery

In [ ]:
import json

all_texts = legal_terms['설명'].to_list()
metadatas = [ {'Term': row['용어명'] } for idx, row in legal_terms.iterrows()]
table.add_texts(all_texts, metadatas=metadatas)
# table.add_texts(all_texts)


## Vector Search
### Search with string

In [ ]:
import time
s = time.time()
query = "가등기권리자란?"

docs = table.similarity_search(query, k=5)
for doc in docs:
  print(f" {doc.metadata['Term']} - {doc.page_content}" )

e = time.time() - s
print(e)

### Search with vector
* https://api.python.langchain.com/en/stable/vectorstores/langchain_community.vectorstores.bigquery_vector_search.BigQueryVectorSearch.html#langchain_community.vectorstores.bigquery_vector_search.BigQueryVectorSearch.similarity_search

In [ ]:
query_vector = embedding.embed_query(query)
docs = table.similarity_search_by_vector(query_vector, k=5)
for doc in docs:
  print(f" {doc.metadata['Term']} - {doc.page_content}" )

### Search with relevant score

In [ ]:
tuples = table.similarity_search_with_relevance_scores(query, k=5)
context ={}

for tp in tuples:
    context[tp[1]] = tp[0].metadata["Term"] + " : " + tp[0].page_content
context

### Max marginal relavant search
* https://api.python.langchain.com/en/stable/vectorstores/langchain_community.vectorstores.bigquery_vector_search.BigQueryVectorSearch.html#langchain_community.vectorstores.bigquery_vector_search.BigQueryVectorSearch.max_marginal_relevance_search

In [ ]:
docs = table.max_marginal_relevance_search(query= query,
                                           k=5,
                                           fetch_k = 30,
                                           lambda_mult = 0.5,
                                           brute_force = True
                                           )
for doc in docs:
  print(doc.page_content)